In [25]:
import yfinance
import json
import jinja2
# For DataFrame
import pandas
import numpy as np
import matplotlib.pyplot as plt
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

def make_pretty(styler):
    # Column formatting
    styler.format({'Forward Earnings Per Share (EPS)': '${:.2f}', 'Forward Price to Earnings Ratio (P/E)': '{:.2f}', 'Price/Earnings-to-Growth Ratio (PEG)': '{:.2f}',
                   'FCFY': '{:.2f}%', 'Price to Book Ratio (P/B)' : '{:.2f}', 'Return on Equity (ROE)' : '{:.2f}', '12-Month Trailing Price-to-Sales Ratio (P/S)': '{:.2f}',
                   'Dividend Payout Ratio (DPR)': '{:.2f}%', 'Dividend Yield (DY)': '{:.2f}%', 'Current Ratio (CR)' : '{:.2f}', 'Beta': '{:.2f}', '52-Week Low': '${:.2f}',
                   'Price': '${:.2f}', '52-Week High': '${:.2f}', '52w Range': '{:.2f}%'
                  })
    # Set the bar visualization
    #styler.bar(subset = ['52w Range'], align = "mid", color = ["salmon", "cornflowerblue"])

    # Grid
    styler.set_properties(**{'border': '0.1px solid black'})

    # Set background gradients
    styler.background_gradient(subset=['Forward Earnings Per Share (EPS)'], cmap='Greens')
    styler.background_gradient(subset=['Forward Price to Earnings Ratio (P/E)'], cmap='Greens')
    styler.background_gradient(subset=['Price/Earnings-to-Growth Ratio (PEG)'], cmap='Greens')
    #styler.background_gradient(subset=['FCFY'], cmap='Greens')
    styler.background_gradient(subset=['Price to Book Ratio (P/B)'], cmap='Greens')
    styler.background_gradient(subset=['Return on Equity (ROE)'], cmap='Greens')
    styler.background_gradient(subset=['12-Month Trailing Price-to-Sales Ratio (P/S)'], cmap='Greens')
    styler.background_gradient(subset=['Dividend Payout Ratio (DPR)'], cmap='Greens')
    styler.background_gradient(subset=['Dividend Yield (DY)'], cmap='Greens')
    #styler.background_gradient(subset=['Current Ratio (CR)'], cmap='Greens')

    # No index
    styler.hide(axis='index')

    # Tooltips
    '''
    styler.set_tooltips(
        ttips, css_class='tt-add',
        props=[
            ('visibility', 'hidden'),
            ('position', 'absolute'),
            ('background-color', 'salmon'),
            ('color', 'black'),
            ('z-index', 1),
            ('padding', '3px 3px'),
            ('margin', '2px')
        ]
    )
    '''
    # Left text alignment for some columns
    styler.set_properties(subset=['Symbol', 'Name', 'Industry'], **{'text-align': 'left'})
    return styler


def populate_tt(df, tt_data, col_name):
    stats = df[col_name].describe()

    per25 = round(stats.loc['25%'], 2)
    per50 = round(stats.loc['50%'], 2)
    per75 = round(stats.loc['75%'], 2)

    # Get position based on the column name
    pos = df.columns.to_list().index(col_name)

    for index, row in df.iterrows():
        pe = row[col_name]
        if pe == stats.loc['min']:
            tt_data[index][pos] = 'Lowest'
        elif pe == stats.loc['max']:
            tt_data[index][pos] = 'Hightest'
        elif pe <= per25:
            tt_data[index][pos] = '25% of companies under {}'.format(per25)
        elif pe <= per50:
            tt_data[index][pos] = '50% of companies under {}'.format(per50)
        elif pe <= per75:
            tt_data[index][pos] = '75% of companies under {}'.format(per75)
        else:
            tt_data[index][pos] = '25% of companies over {}'.format(per75)
def fetch_stock_info(stocks):

    stock_data = {
        'Symbol': [],
        'Name': [],
        'Industry': [],
        'Forward Earnings Per Share (EPS)': [],
        'Forward Price to Earnings Ratio (P/E)': [],
        'Price/Earnings-to-Growth Ratio (PEG)': [],
        # 'Free Cash Flow Yield (FCFY)': []],
        'Price to Book Ratio (P/B)': [],
        'Return on Equity (ROE)': [],
        '12-Month Trailing Price-to-Sales Ratio (P/S)': [],
        'Dividend Payout Ratio (DPR)': [],
        'Dividend Yield (DY)': [],
        # 'Current Ratio (CR)': []],
        'Beta': [],
        'Price': [],
        '52-Week Low': [],
        '52-Week High': []
    }

    for ticker in stocks:
        # GET Yahoo Finance TICKER INFO
        yfTicker = yfinance.Ticker(ticker)
        stock_info = yfTicker.info

        stock_data['Symbol'].append(stock_info['symbol'])
        stock_data['Name'].append(stock_info['shortName'])
        stock_data['Industry'].append(stock_info['industry'])
        stock_data['Forward Earnings Per Share (EPS)'].append(stock_info['forwardEps'])
        stock_data['Forward Price to Earnings Ratio (P/E)'].append(stock_info['forwardPE'])
        stock_data['Price/Earnings-to-Growth Ratio (PEG)'].append(stock_info['pegRatio'])
        #stock_data['Free Cash Flow Yield (FCFY)'].append(stock_info['freeCashflow'])
        stock_data['Price to Book Ratio (P/B)'].append(stock_info['priceToBook'])
        stock_data['Return on Equity (ROE)'].append(stock_info['returnOnEquity'])
        stock_data['12-Month Trailing Price-to-Sales Ratio (P/S)'].append(stock_info['priceToSalesTrailing12Months'])
        stock_data['Dividend Payout Ratio (DPR)'].append(stock_info['payoutRatio'])
        stock_data['Dividend Yield (DY)'].append(stock_info['dividendYield'])
        #stock_data['Current Ratio (CR)'].append(stock_info['currentRatio'])
        stock_data['Beta'].append(stock_info['beta'])
        stock_data['Price'].append(stock_info['currentPrice'])
        stock_data['52-Week Low'].append(stock_info['fiftyTwoWeekLow'])
        stock_data['52-Week High'].append(stock_info['fiftyTwoWeekHigh'])

    # Return a DF using the stock_data dictionary
    return pandas.DataFrame(stock_data)


stocks = ['AAPL', 'GS', 'IBM', 'INTC', 'JNJ', 'JPM', 'MS', 'TRV']
stock_info_dataframe = fetch_stock_info(stocks)
#print(stock_info_dataframe)

# Initialize tool tip data - each column is set to '' for each row
tt_data = [['' for x in range(len(stock_info_dataframe.columns))] for y in range(len(stock_info_dataframe))]

# Gather tool tip data for indicators
#populate_tt(stock_info_dataframe, tt_data, 'Symbol')
#populate_tt(stock_info_dataframe, tt_data, 'Name')
#populate_tt(stock_info_dataframe, tt_data, 'Industry')


# Create a tool tip DF
#ttips = stock_info_dataframe.DataFrame(data=tt_data, columns=stock_info_dataframe.columns, index=stock_info_dataframe.index)

# Add table caption and styles to DF
stock_info_dataframe.style.pipe(make_pretty).set_caption('Fundamental Indicators').set_table_styles(
    [{'selector': 'th.col_heading', 'props': 'text-align: center'},
     {'selector': 'caption', 'props': [('text-align', 'center'),
                                       ('font-size', '11pt'), ('font-weight', 'bold')]}])


Symbol,Name,Industry,Forward Earnings Per Share (EPS),Forward Price to Earnings Ratio (P/E),Price/Earnings-to-Growth Ratio (PEG),Price to Book Ratio (P/B),Return on Equity (ROE),12-Month Trailing Price-to-Sales Ratio (P/S),Dividend Payout Ratio (DPR),Dividend Yield (DY),Beta,Price,52-Week Low,52-Week High
AAPL,Apple Inc.,Consumer Electronics,$7.16,24.11,2.40,36.02,1.54,6.91,0.15%,0.01%,1.29,$172.62,$154.15,$199.62
GS,"Goldman Sachs Group, Inc. (The)",Capital Markets,$39.10,9.90,1.16,1.25,0.07,2.79,0.46%,0.03%,1.42,$387.21,$289.36,$397.49
IBM,International Business Machines,Information Technology Services,$10.66,17.92,3.76,7.76,0.34,2.83,0.81%,0.03%,0.71,$191.07,$120.55,$199.18
INTC,Intel Corporation,Semiconductors,$2.24,19.04,0.74,1.71,0.02,3.32,1.85%,0.01%,1.00,$42.64,$26.86,$51.28
JNJ,Johnson & Johnson,Drug Manufacturers - General,$11.04,14.33,3.18,5.54,0.18,4.48,0.90%,0.03%,0.53,$158.18,$144.95,$175.97
JPM,JP Morgan Chase & Co.,Banks - Diversified,$16.23,11.73,9.82,1.82,0.16,3.75,0.25%,0.02%,1.13,$190.30,$123.11,$191.73
MS,Morgan Stanley,Capital Markets,$7.35,12.02,1.73,1.59,0.09,2.70,0.63%,0.04%,1.41,$88.38,$69.42,$95.57
TRV,"The Travelers Companies, Inc.",Insurance - Property & Casualty,$20.70,10.67,0.78,2.02,0.13,1.22,0.31%,0.02%,0.59,$220.93,$157.33,$224.00
